In [25]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-125705
Azure region: southcentralus
Subscription id: ef22e2eb-a072-4db6-86df-7ea6643b1b5f
Resource group: aml-quickstarts-125705


In [26]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "akcomputecluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_v2',
                                                           max_nodes=10)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [23]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
    "--C": uniform(0.0, 1.0),
    "--max_iter": choice(100, 200, 400, 800)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
pip_packages = [
                'azureml-core==1.17.0', 'azureml-telemetry==1.17.0', 'azureml-dataprep==2.4.2',
                'joblib==0.14.1', 'pandas==1.0.0', 'sklearn-pandas==2.0.2' 
               ]

est = SKLearn(source_directory='.',
              compute_target=compute_target,
              entry_script='train.py',
              pip_packages=pip_packages,
              conda_packages=['scikit-learn==0.22.2.post1'])

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4)

WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['joblib', 'scikit-learn']. We cannot guarantee image build will succeed.


In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [11]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [24]:
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

print(best_run.get_details()['runId'])
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_metrics())

parameters = best_run.get_details()['runDefinition']['arguments']

NameError: name 'hyperdrive_run' is not defined

# AutoML Experiment

In [4]:
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

from azureml.core import Dataset

# create a FileDataset from image and label files behind public web urls
web_path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

ds = Dataset.Tabular.from_delimited_files(path=web_path)

In [5]:
from train import clean_data

x, y = clean_data(ds)

In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=10,
    compute_target=compute_target)

In [14]:
# Submit your automl run
automl_run = exp.submit(automl_config)
automl_run

Running on remote.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_d3455a78-c6c1-4da1-b5f2-33b130fd24f9,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
from azureml.widgets import RunDetails

RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [16]:
automl_run.wait_for_completion(show_output=False)

{'runId': 'AutoML_d3455a78-c6c1-4da1-b5f2-33b130fd24f9',
 'target': 'akcomputecluster',
 'status': 'Completed',
 'startTimeUtc': '2020-11-11T21:14:20.639004Z',
 'endTimeUtc': '2020-11-11T21:57:48.47489Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '10',
  'target': 'akcomputecluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"ef22e2eb-a072-4db6-86df-7ea6643b1b5f","resource_group":"aml-quickstarts-125705","workspace_name":"quick-starts-ws-125705","region":"southcentralus","compute_target":"akcomputecluster","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":10,

In [21]:
# Retrieve and save your best automl model.
best_run, best_model = automl_run.get_output()
best_run.register_model(model_name= 'best_model_automl.pkl', model_path= './outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-125705', subscription_id='ef22e2eb-a072-4db6-86df-7ea6643b1b5f', resource_group='aml-quickstarts-125705'), name=best_model_automl.pkl, id=best_model_automl.pkl:1, version=1, tags={}, properties={})